In [4]:
import os
import mlflow
import mlflow.pyfunc
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Distutils was imported before Setuptools")
warnings.filterwarnings("ignore", category=UserWarning, message="Setuptools is replacing distutils")

# Set the tracking URI using an absolute path
mlflow.set_tracking_uri("../mlruns")
print("MLflow Tracking URI:", mlflow.get_tracking_uri())

# Set the experiment name
mlflow.set_experiment("Guerry")

# Start an MLflow run using context manager
with mlflow.start_run() as run:
    print(f"Active Run ID: {run.info.run_id}")
    print(f"Artifact Location: {run.info.artifact_uri}")
    
    # Fit OLS regression using ln
    data = sm.datasets.get_rdataset("Guerry", "HistData").data
    res = smf.ols('Lottery ~ Literacy + np.log(Pop1831)', data=data).fit()
    
    # Log parameters
    mlflow.log_param("formula", 'Lottery ~ Literacy + np.log(Pop1831)')
    
    # Log metrics
    mlflow.log_metric("r_squared", res.rsquared)
    mlflow.log_metric("adj_r_squared", res.rsquared_adj)

    # Log the model summary as an artifact
    with open("model_summary.txt", "w") as f:
        f.write(str(res.summary()))
    mlflow.log_artifact("model_summary.txt")
    
    # Register the model
    model_uri = f"runs:/{run.info.run_id}/ols_model"
    model_name = "OLSRegressionModel"
    registered_model = mlflow.register_model(model_uri=model_uri, name=model_name)

print("MLflow run completed and model registered.")


MLflow Tracking URI: ../mlruns
Active Run ID: f8c8afbf8fdf4613affe00aff1cd2e54
Artifact Location: file:///c:/Users/diego/OneDrive/Escritorio/Github/statsmodels_learning/notebooks/../mlruns/446769622520076732/f8c8afbf8fdf4613affe00aff1cd2e54/artifacts
MLflow run completed and model registered.


Registered model 'OLSRegressionModel' already exists. Creating a new version of this model...
Created version '2' of model 'OLSRegressionModel'.


In [2]:
print("MLflow Tracking URI:", mlflow.get_tracking_uri())


MLflow Tracking URI: ../mlruns


In [7]:
from pprint import pprint

# Fetch the latest run
latest_run = mlflow.search_runs(experiment_ids=['0']).iloc[0]

print("Run ID:", latest_run['run_id'])

print("\nParameters:")
params = {k.split('params.')[-1]: v for k, v in latest_run.items() if k.startswith('params.')}
pprint(params)

print("\nMetrics:")
metrics = {k.split('metrics.')[-1]: v for k, v in latest_run.items() if k.startswith('metrics.')}
pprint(metrics)

print("\nArtifacts:")
client = mlflow.tracking.MlflowClient()
artifacts = client.list_artifacts(latest_run['run_id'])
for artifact in artifacts:
    print(artifact.path)

# If you want to view the content of a specific artifact (e.g., model_summary.txt):
if any(artifact.path == "model_summary.txt" for artifact in artifacts):
    local_path = client.download_artifacts(latest_run['run_id'], "model_summary.txt")
    with open(local_path, "r") as f:
        print("\nModel Summary:")
        print(f.read())
else:
    print("\nNo model_summary.txt found in artifacts.")

Run ID: 25894ca252764dfd8bbde0b562c2428f

Parameters:
{'formula': 'Lottery ~ Literacy + np.log(Pop1831)'}

Metrics:
{'adj_r_squared': np.float64(0.3327711079168274),
 'r_squared': np.float64(0.3484706112599609)}

Artifacts:
model_summary.txt
ols_model

Model Summary:
                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.333
Method:                 Least Squares   F-statistic:                     22.20
Date:                Fri, 28 Jun 2024   Prob (F-statistic):           1.90e-08
Time:                        01:31:28   Log-Likelihood:                -379.82
No. Observations:                  86   AIC:                             765.6
Df Residuals:                      83   BIC:                             773.0
Df Model:                           2                                         
Covariance Type:     

In [3]:
from mlflow.tracking import MlflowClient

# Instantiate the MLflow client
client = MlflowClient()

# Retrieve all experiments
all_experiments = client.search_experiments()


In [4]:
all_experiments

[<Experiment: artifact_location='file:///c:/Users/diego/OneDrive/Escritorio/Github/statsmodels_learning/notebooks/mlruns/0', creation_time=1719551799698, experiment_id='0', last_update_time=1719551799698, lifecycle_stage='active', name='Default', tags={}>]